# Basic setup for fetching data from the facebook api

### Remember to add an active access token below and adjust the parameters and fields if necessary

In [8]:

# must be specified to be able to use methods of package
import sys
sys.path.append('../src') 
# import packages
from political_ads.helper import *
import pandas as pd
from political_ads.api_request import get_json_response



In [2]:

# insert your access token here (expires every 2 hours)
access_token = "EAADqFZC76BDMBAKhvS0dJ4wGOkX0mF14FZCKfF9bYTzIImcuXAS1AWu4X0XIwx0oN5bDpusy7heDf5CbfZBhPCatpLoFtlpslHHFjiRVmF031tKz0QsrOvOhZBMVMbw5f36OTeDTZBHMJJkDkDgZC5Y2TAjLHNUvt4DZCBjnzSIJFrJCzi2Co8myZC30kFkfSf9KdFRAmfB0aOgAAZBwZBbj4h"
# get_new_access_token: https://developers.facebook.com/tools/explorer/


# PARAMETERS NECESSARY
ad_type = "POLITICAL_AND_ISSUE_ADS"
ad_reached_countries = ['US'] # Facebook delivered the ads in these countries. Provided as ISO country codes.
search_terms = "climate change" # The terms to search for in your query. We treat a blank space as a logical AND and search for both terms and no other operators. The limit of your string is 100 characters or less.
limit = 1700
start_date = "2020-08-00"
end_date = "2020-11-03"


# FIELDS to specify your results
fields = ["id", "page_name", "ad_creative_body", "impressions", "delivery_by_region", "demographic_distribution", "ad_delivery_start_time", "bylines", "spend"]

# see here for more parameters and fields: https://www.facebook.com/ads/library/api/?source=archive-landing-page 



input_url = f"https://graph.facebook.com/v12.0/ads_archive?search_terms={search_terms}&ad_type={ad_type}&ad_reached_countries={ad_reached_countries}&fields={fields}&limit={limit}&ad_delivery_date_min={start_date}&ad_delivery_date_max={end_date}"

response = get_json_response(input_url, access_token)

data = pd.DataFrame(response["data"])


In [3]:
import time 

#https://developers.facebook.com/docs/graph-api/results

def larger_data_import(iterations, sleep_time):
    counter = 0
    for i in range(iterations):

        paging = response["paging"]["next"]
        next_response = get_json_response(paging, access_token)

        new_page = pd.DataFrame(next_response['data'])
        data = data.append(new_page, ignore_index=True)
        time.sleep(sleep_time)
        counter+=1
        print(counter)

#larger_data_import(10, 10)


In [4]:
#Write to CSV
#data.to_csv('test.csv')


In [5]:
#Cleaning the Data // Dropping missing values
data.dropna(inplace=True)

In [10]:
data.drop(['age_distribution', 'gender'], axis=1)

data['age_distribution'] = data['demographic_distribution'].apply(lambda x: get_age_distribution(x, all_ages=True))

data['gender'] = data['demographic_distribution'].apply(lambda x: get_gender_distribution(x))

data['impressions'] = data.impressions.apply(lambda x: transform_range(x))


In [7]:


#https://datascience.stackexchange.com/questions/9616/how-to-create-us-state-choropleth-map
#Convert the demographic data into a heat-map
